# Introduction

We're running a massive final D&D 5e battlefield, and would like to roll the dice!
but having to roll 20d20 to determine if your army of archers hits is a pain
so here is some R code to simulate dice-rolls for large groups of mobs.

It shows you exactly what was rolled so you can still cheer on your army of champions!
(except for individual damage dice, but I don't care too much about that)


In [1]:
library(glue)

# functions

In [2]:
#' Do a single dice roll
#'
#' this allows you to roll e.g. 3d8 by calling `roll_dice(3, 8)`.
#' @return the total of the two dice
roll_dice <- function(n = 1, d = 20) {
  sum(sample(seq_len(d), size = n, replace = TRUE))
}

In [3]:
#' Roll a d20
#'
#' This is the special case for the d20, that allows you to roll with advantage/disadvantage
#' It returns only the roll, but can show a message with roll results if desired.
d20 <- function(advantage = FALSE, disadvantage = FALSE, verbose = advantage | disadvantage) {
  # roll a d20
  roll1 <- roll_dice(1, 20)

  if (advantage & disadvantage) {
    warning("You have specified both advantage and disadvantage, they cancel out.")
    advantage <- disadvantage <- FALSE
  }

  # roll a second time
  if (advantage | disadvantage) {
    roll2 <- roll_dice(1, 20)
  } else {
    roll2 <- NA
  }

  if (advantage) {
    roll <- max(roll1, roll2)
  } else if (disadvantage) {
    roll <- min(roll1, roll2)
  } else {
    roll <- roll1
  }

  if (verbose) {
    msg <- paste0("with advantage"[advantage], "with disadvantage"[disadvantage])
    ## message(glue::glue("Rolling a d20 {msg}"))
    if (advantage | disadvantage) {
      msg2 <- glue::glue(" and a {roll2}")
    } else {
      msg2 <- ""
    }
    message(glue::glue("Rolled a: {roll1}{msg2}"))
  }

  roll
}

In [4]:
#' Attack with a group of mobs
#'
#' For example, a group of 20 3rd-level rangers with archery have a `to_hit` of 8
#' They deal 1d8 + 4 damage
#' Specify the defender's AC to see if they hit and to total the damage.
attack <- function(n_mobs = 20,
                   to_hit = +8,
                   dmg_die_n = 1,
                   dmg_die = 8,
                   dmg_bonus = +4,
                   AC = 15,
                   advantage = FALSE,
                   disadvantage = FALSE) {
  # do n_mobs d20 rolls
  rolls <- replicate(n_mobs, d20(advantage = advantage,
                                 disadvantage = disadvantage))
  crit_hit <- rolls == 20
  crit_miss <- rolls == 1
  crit_n <- sum(crit_hit)
  crit_miss_n <- sum(crit_miss)
  hits <- rolls + to_hit >= AC # for AC it's "equals or exceeds" PHB 194
  hits[crit_hit] <- TRUE # a critical hit ALWAYS hits, regardless of the AC
  hits[crit_miss] <- FALSE # a critical miss ALWAYS misses, regardless of the AC
  hit_n <- sum(hits)

  # do damage rolls
  dmg_n <- hit_n + crit_n # we add damage dice rolls for critical hits
  dmg <- replicate(dmg_n, roll_dice(dmg_die_n, dmg_die) + dmg_bonus)
  tot <- sum(dmg)
  list(d20 = rolls,
       hit = rolls + to_hit,
       crit_n = crit_n,
       crit_miss_n = crit_miss_n,
       hit_n = hit_n,
       dmgn = dmg_n,
       dmg = dmg,
       tot = tot)
}

In [5]:
#' Roll saving throws for a group of mobs
#'
#' For example, with a group of 20 archers who are proficient in DEX saves,
#' they roll d20+6 against the DC.
save <- function(n_mobs = 20,
                 skill_bonus = +6,
                 DC = 15,
                 advantage = FALSE,
                 disadvantage = FALSE) {
  rolls <- replicate(n_mobs, d20(advantage = advantage,
                                 disadvantage = disadvantage))
  success <- rolls + skill_bonus >= DC # equal or exceed = PHB page 7 (unless contested roll)
  success_n <- sum(success)
  list(d20 = rolls, check = rolls + skill_bonus, success_n = success_n)
}

# a synonym for a skill save
#' Roll skill checks for a group of mobs
check <- save

### Some examples

In [6]:
# attack with 15 human hunters with a shortsword.
attack(15, +6, 1, 8, +4, AC = 15)

$d20
 [1] 14 18 20 12  8 19 15 18  6  4 18  1  1 14 15

$hit
 [1] 20 24 26 18 14 25 21 24 12 10 24  7  7 20 21

$crit_n
[1] 1

$crit_miss_n
[1] 2

$hit_n
[1] 10

$dmgn
[1] 11

$dmg
 [1] 12 10 12 10  5 10  8  9  8  5 10

$tot
[1] 99

In [7]:
# they can also attack with their off-hand using their bonus action, but for
# the damage we do not add the ability modifier
attack(15, +6, 1, 8, +0, AC = 15)

$d20
 [1] 16 15  3  2  2 11  7 18  6 11 14 14 16  5  7

$hit
 [1] 22 21  9  8  8 17 13 24 12 17 20 20 22 11 13

$crit_n
[1] 0

$crit_miss_n
[1] 0

$hit_n
[1] 8

$dmgn
[1] 8

$dmg
[1] 6 5 4 8 7 7 7 4

$tot
[1] 48

In [8]:
# our hunters are colossus slayers, so they deal an extra 1d8 damage if the
# target's HP is below it's max HP: (only once per turn, only for first attack)
attack(15, +6, 2, 8, +4, AC = 15)

$d20
 [1] 16 20 11 14  8 11 11 15 10 18 10  8 20  6  2

$hit
 [1] 22 26 17 20 14 17 17 21 16 24 16 14 26 12  8

$crit_n
[1] 2

$crit_miss_n
[1] 0

$hit_n
[1] 11

$dmgn
[1] 13

$dmg
 [1] 14 15 11 15 19 15 14 12 12 17 11 12 12

$tot
[1] 179

In [9]:
# attack with 19 archers that have a +8 bonus to hit. They are also colossus
# slayers so they roll 2d8+4 damage against an AC of 15, with advantage
attack(19, +8, 2, 8, +4, AC = 15, adv = TRUE)

Rolled a: 8 and a 18

Rolled a: 3 and a 5

Rolled a: 9 and a 14

Rolled a: 14 and a 5

Rolled a: 18 and a 7

Rolled a: 9 and a 20

Rolled a: 5 and a 3

Rolled a: 6 and a 19

Rolled a: 14 and a 10

Rolled a: 7 and a 17

Rolled a: 7 and a 13

Rolled a: 9 and a 17

Rolled a: 20 and a 11

Rolled a: 19 and a 20

Rolled a: 11 and a 8

Rolled a: 13 and a 19

Rolled a: 8 and a 10

Rolled a: 9 and a 20

Rolled a: 17 and a 11



$d20
 [1] 18  5 14 14 18 20  5 19 14 17 13 17 20 20 11 19 10 20 17

$hit
 [1] 26 13 22 22 26 28 13 27 22 25 21 25 28 28 19 27 18 28 25

$crit_n
[1] 4

$crit_miss_n
[1] 0

$hit_n
[1] 17

$dmgn
[1] 21

$dmg
 [1] 17 13  9 14 13  9 12  8 18 14 18 14 20  7 14 18 17 11 10 11 10

$tot
[1] 277

In [10]:
# Do a stealth check for 15 human ranger hunters, who have +6 on stealth
check(15, +6, DC = 14)

$d20
 [1] 18 11 10  9 14 20 19  7 13  8 16  3  2  3  8

$check
 [1] 24 17 16 15 20 26 25 13 19 14 22  9  8  9 14

$success_n
[1] 11